#  Baseline Neural Network Model Performance

In [0]:
import numpy as np
import pandas as pd
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split

# Next, we can initialize the random number generator to ensure that we always get the same results when executing this code. This will help if we are debugging:

In [0]:
seed = 7
np.random.seed(seed)

# Load the dataset and split the columns into 60 input variables (X) and 1 output variable (Y)

In [12]:
from google.colab import files
uploaded = files.upload()
df=pd.read_csv("sonar.csv")

Saving sonar.csv to sonar (1).csv


In [13]:
df.head()

,0.0200,0.0371,0.0428,0.0207,0.0954,0.0986,0.1539,0.1601,0.3109,0.2111,...,0.0027,0.0065,0.0159,0.0072,0.0167,0.0180,0.0084,0.0090,0.0032,R
0,0.0453,0.0523,0.0843,0.0689,0.1183,0.2583,0.2156,0.3481,0.3337,0.2872,...,0.0084,0.0089,0.0048,0.0094,0.0191,0.0140,0.0049,0.0052,0.0044,R
1,0.0262,0.0582,0.1099,0.1083,0.0974,0.2280,0.2431,0.3771,0.5598,0.6194,...,0.0232,0.0166,0.0095,0.0180,0.0244,0.0316,0.0164,0.0095,0.0078,R
2,0.0100,0.0171,0.0623,0.0205,0.0205,0.0368,0.1098,0.1276,0.0598,0.1264,...,0.0121,0.0036,0.0150,0.0085,0.0073,0.0050,0.0044,0.0040,0.0117,R
3,0.0762,0.0666,0.0481,0.0394,0.0590,0.0649,0.1209,0.2467,0.3564,0.4459,...,0.0031,0.0054,0.0105,0.0110,0.0015,0.0072,0.0048,0.0107,0.0094,R
4,0.0286,0.0453,0.0277,0.0174,0.0384,0.0990,0.1201,0.1833,0.2105,0.3039,...,0.0045,0.0014,0.0038,0.0013,0.0089,0.0057,0.0027,0.0051,0.0062,R


In [14]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 207 entries, 0 to 206
Data columns (total 61 columns):
0.0200    207 non-null float64
0.0371    207 non-null float64
0.0428    207 non-null float64
0.0207    207 non-null float64
0.0954    207 non-null float64
0.0986    207 non-null float64
0.1539    207 non-null float64
0.1601    207 non-null float64
0.3109    207 non-null float64
0.2111    207 non-null float64
0.1609    207 non-null float64
0.1582    207 non-null float64
0.2238    207 non-null float64
0.0645    207 non-null float64
0.0660    207 non-null float64
0.2273    207 non-null float64
0.3100    207 non-null float64
0.2999    207 non-null float64
0.5078    207 non-null float64
0.4797    207 non-null float64
0.5783    207 non-null float64
0.5071    207 non-null float64
0.4328    207 non-null float64
0.5550    207 non-null float64
0.6711    207 non-null float64
0.6415    207 non-null float64
0.7104    207 non-null float64
0.8080    207 non-null float64
0.6791    207 non-null floa

In [15]:
df.describe()

,0.0200,0.0371,0.0428,0.0207,0.0954,0.0986,0.1539,0.1601,0.3109,0.2111,...,0.0232,0.0027,0.0065,0.0159,0.0072,0.0167,0.0180,0.0084,0.0090,0.0032
count,207.000000,207.000000,207.000000,207.000000,207.000000,207.000000,207.000000,207.000000,207.000000,207.000000,...,207.000000,207.000000,207.000000,207.000000,207.000000,207.000000,207.000000,207.000000,207.000000,207.000000
mean,0.029208,0.038443,0.043837,0.054053,0.075105,0.104599,0.121591,0.134677,0.177361,0.208245,...,0.016034,0.013472,0.010729,0.010917,0.009300,0.008181,0.007771,0.007947,0.007936,0.006523
std,0.023038,0.033040,0.038521,0.046583,0.055669,0.059247,0.061897,0.085340,0.118311,0.134741,...,0.012027,0.009628,0.007071,0.007310,0.007103,0.005719,0.005756,0.006485,0.006196,0.005038
min,0.001500,0.000600,0.001500,0.005800,0.006700,0.010200,0.003300,0.005500,0.007500,0.011300,...,0.000000,0.000800,0.000500,0.001000,0.000600,0.000400,0.000300,0.000300,0.000100,0.000600
25%,0.013300,0.016400,0.018900,0.024450,0.037700,0.066950,0.080600,0.080350,0.096750,0.111150,...,0.008350,0.007350,0.005050,0.005350,0.004100,0.004400,0.003700,0.003600,0.003650,0.003100
50%,0.022800,0.030800,0.034200,0.044100,0.062000,0.092100,0.105600,0.111900,0.152200,0.181000,...,0.013800,0.011500,0.009600,0.009300,0.007500,0.006800,0.005900,0.005800,0.006300,0.005300
75%,0.035800,0.048100,0.058200,0.065700,0.101050,0.134150,0.153050,0.169800,0.231500,0.269000,...,0.020700,0.016750,0.014900,0.014450,0.012100,0.010350,0.010350,0.010400,0.010350,0.008550
max,0.137100,0.233900,0.305900,0.426400,0.401000,0.382300,0.372900,0.459000,0.682800,0.710600,...,0.100400,0.070900,0.039000,0.035200,0.044700,0.039400,0.035500,0.044000,0.036400,0.043900


In [0]:
data=df.values

In [0]:
# split into input (X) and output (Y) variables
X = data[:,0:60].astype(float)
Y = data[:,60]

In [18]:
X

array([[0.0453, 0.0523, 0.0843, ..., 0.0049, 0.0052, 0.0044],
       [0.0262, 0.0582, 0.1099, ..., 0.0164, 0.0095, 0.0078],
       [0.01  , 0.0171, 0.0623, ..., 0.0044, 0.004 , 0.0117],
       ...,
       [0.0522, 0.0437, 0.018 , ..., 0.0138, 0.0077, 0.0031],
       [0.0303, 0.0353, 0.049 , ..., 0.0079, 0.0036, 0.0048],
       [0.026 , 0.0363, 0.0136, ..., 0.0036, 0.0061, 0.0115]])

In [19]:
Y

array(['R', 'R', 'R', 'R', 'R', 'R', 'R', 'R', 'R', 'R', 'R', 'R', 'R',
       'R', 'R', 'R', 'R', 'R', 'R', 'R', 'R', 'R', 'R', 'R', 'R', 'R',
       'R', 'R', 'R', 'R', 'R', 'R', 'R', 'R', 'R', 'R', 'R', 'R', 'R',
       'R', 'R', 'R', 'R', 'R', 'R', 'R', 'R', 'R', 'R', 'R', 'R', 'R',
       'R', 'R', 'R', 'R', 'R', 'R', 'R', 'R', 'R', 'R', 'R', 'R', 'R',
       'R', 'R', 'R', 'R', 'R', 'R', 'R', 'R', 'R', 'R', 'R', 'R', 'R',
       'R', 'R', 'R', 'R', 'R', 'R', 'R', 'R', 'R', 'R', 'R', 'R', 'R',
       'R', 'R', 'R', 'R', 'R', 'M', 'M', 'M', 'M', 'M', 'M', 'M', 'M',
       'M', 'M', 'M', 'M', 'M', 'M', 'M', 'M', 'M', 'M', 'M', 'M', 'M',
       'M', 'M', 'M', 'M', 'M', 'M', 'M', 'M', 'M', 'M', 'M', 'M', 'M',
       'M', 'M', 'M', 'M', 'M', 'M', 'M', 'M', 'M', 'M', 'M', 'M', 'M',
       'M', 'M', 'M', 'M', 'M', 'M', 'M', 'M', 'M', 'M', 'M', 'M', 'M',
       'M', 'M', 'M', 'M', 'M', 'M', 'M', 'M', 'M', 'M', 'M', 'M', 'M',
       'M', 'M', 'M', 'M', 'M', 'M', 'M', 'M', 'M', 'M', 'M', 'M

# The output variable is string values. We must convert them into integer values 0 and 1.

In [0]:
labelEncode = LabelEncoder()
Y =  labelEncode.fit_transform(Y)

In [21]:
Y

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0])

# Creating a baseline model

In [0]:
#Initializations define the way to set the initial random weights of Keras layers
from keras import initializers
initial = initializers.random_normal()
def create_baseline():
    # create model
    model = Sequential()
    model.add(Dense(60, input_dim=60, kernel_initializer=initial, activation='relu'))
    model.add(Dense(1, kernel_initializer=initial, activation='sigmoid'))
    # Compile model
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

### Now it is time to evaluate this model using stratified cross validation in the scikit-learn framework.

In [23]:
#To use Keras models with scikit-learn, we must use the KerasClassifier wrapper.
estimator = KerasClassifier(build_fn=create_baseline, epochs=100, batch_size=5, verbose=0)
kFolds = StratifiedKFold(n_splits=10, shuffle=True, random_state=seed)
results = cross_val_score(estimator, X, Y, cv=kFolds)
print("Results: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))

Results: 82.09% (7.59%)


# Re-Run The Baseline Model With Data Preparation

In [24]:
# Standardization is important because the data is rescaled such that the mean value for each attribute is 0 and the standard deviation is 1. 
# evaluate baseline model with standardized dataset
np.random.seed(seed)
estimators = []
estimators.append(('standardize', StandardScaler()))
estimators.append(('mlp', KerasClassifier(build_fn=create_baseline, epochs=100, batch_size=5, verbose=0)))
pipeline = Pipeline(estimators)
kFolds = StratifiedKFold(n_splits=10, shuffle=True, random_state=seed)
results = cross_val_score(pipeline, X, Y, cv=kFolds)
print("Standardized: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))

Standardized: 86.02% (3.24%)


# Tuning Layers and Number of Neurons in The Model

### 4.1. Evaluate a Smaller Network

In [25]:
def create_smaller():
    # create model
    model = Sequential()
    model.add(Dense(30, input_dim=60, kernel_initializer='normal', activation='relu'))
    model.add(Dense(1, kernel_initializer='normal', activation='sigmoid'))
    # Compile model
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model
estimators = []
estimators.append(('standardize', StandardScaler()))
estimators.append(('mlp', KerasClassifier(build_fn=create_smaller, epochs=100, batch_size=5, verbose=0)))
pipeline = Pipeline(estimators)
kFolds = StratifiedKFold(n_splits=10, shuffle=True, random_state=seed)
results = cross_val_score(pipeline, X,Y, cv=kFolds)
print("Smaller: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))

Smaller: 85.50% (3.70%)


# Step 4.2. Evaluate a Larger Network

In [26]:
def create_larger():
    # create model
    model = Sequential()
    model.add(Dense(60, input_dim=60, kernel_initializer='normal', activation='relu'))
    model.add(Dense(30, kernel_initializer='normal', activation='relu'))
    model.add(Dense(1, kernel_initializer='normal', activation='sigmoid'))
    # Compile model
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model
estimators = []
estimators.append(('standardize', StandardScaler()))
estimators.append(('mlp', KerasClassifier(build_fn=create_larger, epochs=100, batch_size=5, verbose=0)))
pipeline = Pipeline(estimators)
kFolds = StratifiedKFold(n_splits=10, shuffle=True, random_state=seed)
results = cross_val_score(pipeline, X, Y, cv=kFolds)
print("Larger: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))

Larger: 87.45% (4.88%)


# Rewriting the code using the Keras Functional API

In [27]:
import keras
from keras import layers
def kerasApiModel():
    # create model
                    inputs = keras.Input(shape=(60,))
                    a = layers.Dense(60, activation='relu')(inputs)
                    a = layers.Dense(10, activation='relu')(a)
                    outputs = layers.Dense(1, activation='sigmoid')(a)
                    model = keras.Model(inputs, outputs)
                     # Compile model
                    model.compile(loss='binary_crossentropy', optimizer='adam',   metrics=['accuracy'])
                    return model

estimators = []
estimators.append(('standardize', StandardScaler()))
estimators.append(('mlp', KerasClassifier(build_fn=kerasApiModel, epochs=20, batch_size=4, verbose=0)))
pipeline = Pipeline(estimators)
kFolds = StratifiedKFold(n_splits=4, shuffle=True, random_state=seed)
results = cross_val_score(pipeline, X, Y, cv=kFolds)
print("Smaller: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))

Smaller: 87.42% (4.02%)


# Rewriting the code by doing Model Subclassing

In [28]:
import tensorflow as tf
import keras
class MyModel(tf.keras.Model):
    
        def __init__(self):
            
                        super(MyModel, self).__init__()
                        self.dense1 = Dense(60, activation="relu")
                        self.dense2 = Dense(10, activation='relu')
                        self.dense3 = Dense(1, activation='sigmoid')

        def call(self, inputs):

                        x = self.dense1(inputs)
                        x = self.dense2(x)
                        return self.dense3(x)
                        
                    
model = MyModel()
model.compile(loss='binary_crossentropy', optimizer='adam',   metrics=['accuracy'])
model.fit(X,Y, epochs=20, batch_size=4,verbose=False)

In [32]:
# **Build Model from scratch without Scikit Learn**
from keras import optimizers
def final_model():
  network = Sequential()
  network.add(Dense(60, activation='relu',input_shape=(60,)))
  network.add(Dense(30, activation='relu'))
  network.add(Dense(1, activation='sigmoid'))
  network.compile(optimizer = optimizers.Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-8), loss='binary_crossentropy', metrics=["accuracy"])
  return network

model = final_model()
model.fit(X, Y , epochs=100, batch_size=5 , verbose=1)

Epoch 1/100
207/207 [==============================] - 2s 10ms/step - loss: 0.6807 - acc: 0.5652
Epoch 2/100
207/207 [==============================] - 0s 489us/step - loss: 0.6301 - acc: 0.6812
Epoch 3/100
207/207 [==============================] - 0s 477us/step - loss: 0.5984 - acc: 0.7005
Epoch 4/100
207/207 [==============================] - 0s 511us/step - loss: 0.5555 - acc: 0.7246
Epoch 5/100
207/207 [==============================] - 0s 495us/step - loss: 0.5116 - acc: 0.7391
Epoch 6/100
207/207 [==============================] - 0s 474us/step - loss: 0.4989 - acc: 0.7440
Epoch 7/100
207/207 [==============================] - 0s 480us/step - loss: 0.4497 - acc: 0.8116
Epoch 8/100
207/207 [==============================] - 0s 546us/step - loss: 0.4299 - acc: 0.8019
Epoch 9/100
207/207 [==============================] - 0s 495us/step - loss: 0.4163 - acc: 0.8309
Epoch 10/100
207/207 [==============================] - 0s 499us/step - loss: 0.4133 - acc: 0.8164
Epoch 11/100
207/207

  5/207 [..............................] - ETA: 0s - loss: 0.0019 - acc: 1.0000207/207 [==============================] - 0s 422us/step - loss: 0.0048 - acc: 1.0000
Epoch 96/100
207/207 [==============================] - 0s 512us/step - loss: 0.0047 - acc: 1.0000
Epoch 97/100
207/207 [==============================] - 0s 493us/step - loss: 0.0041 - acc: 1.0000
Epoch 98/100
207/207 [==============================] - 0s 456us/step - loss: 0.0045 - acc: 1.0000
Epoch 99/100
207/207 [==============================] - 0s 458us/step - loss: 0.0038 - acc: 1.0000
Epoch 100/100
207/207 [==============================] - 0s 484us/step - loss: 0.0035 - acc: 1.0000
